In [1]:
import os
import re
import urllib.request

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
from rasterio.warp import transform
import pandas as pd
import xarray as xr
from dask.diagnostics import ProgressBar

In [2]:
ref = xr.open_dataset("../../../data_general/proc_sif-optim/covar_nc/sif_refgrid.nc")
ref = ref['SIF']
ref = ref.rename({"lon":"x",
           "lat":"y"})

In [3]:
da = xr.open_rasterio('../../../data_general/proc_sif-optim/myd11a1_2018-0000000000-0000000000.tif',chunks={"band":10})
da = da.rename({'band':'time'})
da.coords['time'] = pd.date_range("2018-01-01", periods=364)

In [4]:
da

<xarray.DataArray (time: 364, y: 1280, x: 1280)>
dask.array<open_rasterio-b4635a878e542f0ffb154b29af402212<this-array>, shape=(364, 1280, 1280), dtype=float64, chunksize=(10, 1280, 1280), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-12-30
  * y        (y) float64 49.38 49.38 49.37 49.36 49.35 ... 37.92 37.91 37.9 37.9
  * x        (x) float64 -124.7 -124.7 -124.7 -124.7 ... -113.3 -113.3 -113.2
Attributes:
    transform:      (0.008983152841195215, 0.0, -124.74006035283676, 0.0, -0....
    crs:            +init=epsg:4326
    res:            (0.008983152841195215, 0.008983152841195215)
    is_tiled:       1
    nodatavals:     (nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n...
    scales:         (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...
    offsets:        (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...
    AREA_OR_POINT:  Area

In [34]:
ref = ref.sel(x=slice(np.min(da['x']), np.max(da['x'])))
ref = ref.sel(y=slice(np.min(da['y']), np.max(da['y'])))

In [38]:
ref['SIF'] = True

dtype('float32')

In [9]:
# regridding step (memory hungry)
result = da.interp(x=ref['x'], y=ref['y'],method='nearest')


/home/sami/miniconda3/envs/sci/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [5]:
with ProgressBar():
    result = result.compute()

NameError: name 'result' is not defined

In [14]:
result

<xarray.DataArray (time: 364, y: 2554, x: 2553)>
array([[[   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        ...,
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan]],

       [[   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        ...,
        [   nan,    nan,    nan, ..., 265.46, 265.46, 265.46],
        [   nan,    nan,    nan, ..., 265.46, 265.46, 265.46],
        [   nan,    nan,    nan, ..., 265.2 , 265.2 , 265.14]],

       [[   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        ...,
...
        ...,
        [   nan,    nan,    nan, ..., 272.24, 272.24, 272.24],
        [   nan,    nan,    nan, ..., 272.24, 272.24, 272.24],
        [   nan,    nan,    nan, ..., 272.5 , 272.5 , 272.3 ]],

       [[   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        ...,
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan]],

       [[   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        [   nan,    nan,    nan, ...,    nan,    nan,    nan],
        ...,
        [272.82, 272.82, 272.82, ...,    nan,    nan,    nan],
        [272.82, 272.82, 272.82, ...,    nan,    nan,    nan],
        [273.28, 273.28, 273.28, ...,    nan,    nan,    nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-12-30
  * x        (x) float64 -124.7 -124.7 -124.7 -124.7 ... -113.3 -113.3 -113.3
  * y        (y) float64 37.9 37.9 37.91 37.91 37.91 ... 49.37 49.38 49.38 49.38
Attributes:
    transform:      (0.008983152841195215, 0.0, -124.74006035283676, 0.0, -0....
    crs:            +init=epsg:4326
    res:            (0.008983152841195215, 0.008983152841195215)
    is_tiled:       1
    nodatavals:     (nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n...
    scales:         (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...
    offsets:        (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...
    AREA_OR_POINT:  Area

In [17]:
result.rename('lst').to_netcdf('../../../data_general/proc_sif-optim/covar_nc/myd11a1_2018-0000000000-0000000000_regrid.nc',
                               format='netcdf4',
                              encoding = {'lst': {'dtype': 'int16', 'scale_factor': 0.1,'zlib':True}})

In [3]:
# import glob
# glob.glob("../../../data_general/proc_sif-optim/",'*')


In [2]:
import fnmatch
flist = []
for file_name in os.listdir("../../../data_general/proc_sif-optim/"):
    if fnmatch.fnmatch(file_name, 'myd11a1_*-*'):
#         print(file_name)
        flist.append(file_name)
    
import re
y = re.search("(\\d{4})",flist[0])
y = y.group()
vec_time = pd.date_range(y+"-01-01", periods=364)

In [3]:
def regrid_lst(fname):
    fpath = "../../../data_general/proc_sif-optim/"+fname
    fout_name = fname.rsplit('.')[0]+"_regrid.nc"
    fout_path = "../../../data_general/proc_sif-optim/covar_nc/"+fout_name
    ref = xr.open_dataset("../../../data_general/proc_sif-optim/covar_nc/sif_refgrid.nc")
    ref = ref['SIF']
    ref = ref.rename({"lon":"x",
           "lat":"y"})
    
    # get the year from the filename 
    y = re.search("(\\d{4})",flist[0])
    y = y.group()
    
    # open w/chunking
    da = xr.open_rasterio(fpath,chunks={"band":40})
    da = da.rename({'band':'time'})
    vec_time = pd.date_range(y+"-01-01", periods=len(da['time']))
    da.coords['time'] = vec_time
    
    # subset reference grid
    ref = ref.sel(x=slice(np.min(da['x']), np.max(da['x'])))
    ref = ref.sel(y=slice(np.min(da['y']), np.max(da['y'])))
    
    result = da.interp(x=ref['x'], y=ref['y'],method='nearest')
    with ProgressBar():
        result = result.compute()
        
    result.rename('lst').to_netcdf(fout_path,
                               format='netcdf4',
                              encoding = {'lst': {'dtype': 'int16', 
                                                  'scale_factor': 0.1,
                                                  'zlib':True}})
    return result


In [14]:
done_list = []
for file_name in os.listdir("../../../data_general/proc_sif-optim/covar_nc/"):
    if fnmatch.fnmatch(file_name, 'myd11a1_*-*'):
#         print(file_name)
        done_list.append(file_name.rsplit("_regrid")[0])
done_list

['myd11a1_2018-0000000000-0000006400',
 'myd11a1_2019-0000002560-0000002560',
 'myd11a1_2018-0000001280-0000001280',
 'myd11a1_2020-0000001280-0000001280',
 'myd11a1_2018-0000002560-0000001280',
 'myd11a1_2018-0000002560-0000005120',
 'myd11a1_2018-0000002560-0000002560',
 'myd11a1_2019-0000000000-0000002560',
 'myd11a1_2020-0000000000-0000002560',
 'myd11a1_2019-0000000000-0000005120',
 'myd11a1_2020-0000002560-0000002560',
 'myd11a1_2018-0000002560-0000003840',
 'myd11a1_2020-0000000000-0000000000',
 'myd11a1_2020-0000001280-0000002560',
 'myd11a1_2020-0000002560-0000003840',
 'myd11a1_2020-0000000000-0000001280',
 'myd11a1_2020-0000001280-0000005120',
 'myd11a1_2020-0000000000-0000005120',
 'myd11a1_2019-0000000000-0000001280',
 'myd11a1_2018-0000000000-0000002560',
 'myd11a1_2018-0000001280-0000006400',
 'myd11a1_2018-0000001280-0000005120',
 'myd11a1_2018-0000000000-0000003840',
 'myd11a1_2019-0000001280-0000005120',
 'myd11a1_2019-0000001280-0000000000',
 'myd11a1_2018-0000000000

In [15]:
name_list = []
for file_name in os.listdir("../../../data_general/proc_sif-optim/"):
    if fnmatch.fnmatch(file_name, 'myd11a1_*-*'):
#         print(file_name)
        name_list.append(file_name.rsplit(".tif")[0])
name_list

['myd11a1_2019-0000000000-0000006400',
 'myd11a1_2020-0000001280-0000003840',
 'myd11a1_2019-0000002560-0000002560',
 'myd11a1_2018-0000000000-0000005120',
 'myd11a1_2020-0000002560-0000003840',
 'myd11a1_2018-0000002560-0000003840',
 'myd11a1_2020-0000000000-0000006400',
 'myd11a1_2020-0000001280-0000000000',
 'myd11a1_2019-0000001280-0000001280',
 'myd11a1_2018-0000001280-0000001280',
 'myd11a1_2020-0000001280-0000006400',
 'myd11a1_2020-0000002560-0000001280',
 'myd11a1_2019-0000002560-0000006400',
 'myd11a1_2019-0000000000-0000002560',
 'myd11a1_2020-0000000000-0000000000',
 'myd11a1_2018-0000000000-0000001280',
 'myd11a1_2020-0000000000-0000003840',
 'myd11a1_2019-0000001280-0000002560',
 'myd11a1_2020-0000000000-0000001280',
 'myd11a1_2019-0000002560-0000005120',
 'myd11a1_2018-0000002560-0000005120',
 'myd11a1_2018-0000000000-0000003840',
 'myd11a1_2018-0000001280-0000000000',
 'myd11a1_2018-0000002560-0000006400',
 'myd11a1_2019-0000000000-0000000000',
 'myd11a1_2018-0000002560

In [16]:
to_do = set(name_list).symmetric_difference(done_list)
to_do = list(to_do)
to_do

['myd11a1_2019-0000002560-0000006400',
 'myd11a1_2018-0000002560-0000006400',
 'myd11a1_2020-0000002560-0000006400']

In [17]:
for i in to_do:
    regrid_lst(i+".tif")

[########################################] | 100% Completed |  0.4s
[                                        ] | 0% Completed |  0.0s

/home/sami/miniconda3/envs/sci/lib/python3.8/site-packages/xarray/core/dataarray.py:2741: SerializationWarning: saving variable lst with floating point data as an integer dtype without any _FillValue to use for NaNs
  return dataset.to_netcdf(*args, **kwargs)


[########################################] | 100% Completed |  0.4s
[                                        ] | 0% Completed |  0.0s

/home/sami/miniconda3/envs/sci/lib/python3.8/site-packages/xarray/core/dataarray.py:2741: SerializationWarning: saving variable lst with floating point data as an integer dtype without any _FillValue to use for NaNs
  return dataset.to_netcdf(*args, **kwargs)


[########################################] | 100% Completed |  0.4s


/home/sami/miniconda3/envs/sci/lib/python3.8/site-packages/xarray/core/dataarray.py:2741: SerializationWarning: saving variable lst with floating point data as an integer dtype without any _FillValue to use for NaNs
  return dataset.to_netcdf(*args, **kwargs)


In [1]:
flist

NameError: name 'flist' is not defined

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42645")
client

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42645")
client

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:42645")
client

In [59]:
def regrid(x, y, dim):
    return xr.apply_ufunc(
        xr.interp,
        x,
        y,
        input_core_dims=[[dim], [dim]],
        dask="parallelized",
        output_dtypes=[float],
    )

In [60]:
regrid

<function __main__.regrid(x, y, dim)>

In [ ]:
result = da.interp(x=ref['x'], y=ref['y'],method='nearest')


In [62]:
res2 = regrid(da)

TypeError: regrid() missing 2 required positional arguments: 'y' and 'dim'

In [67]:
res2

<xarray.DataArray (time: 364, y: 2554, x: 66)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time     (time) datetime64[ns] 2019-01-01 2019-01-02 ... 2019-12-30
  * x        (x) float64 -67.24 -67.24 -67.23 -67.23 ... -66.96 -66.95 -66.95
  * y        (y) float64 37.9 37.9 37.91 37.91 37.91 ... 49.37 49.38 49.38 49.38
Attributes:
    transform:      (0.008983152841195215, 0.0, -67.24788216918739, 0.0, -0.0...
    crs:            +init=epsg:4326
    res:            (0.008983152841195215, 0.008983152841195215)
    is_tiled:       1
    nodatavals:     (nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n...
    scales:         (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1...
    offsets:        (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...
    AREA_OR_POINT:  Area